# Labelme转mask-批量

同济子豪兄 2023-6-8

## 下载样例数据集

In [1]:
!rm -rf Watermelon87_Semantic_Seg_Labelme.zip Watermelon87_Semantic_Seg_Labelme # 删除原有压缩包和文件夹
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/dataset/watermelon/Watermelon87_Semantic_Seg_Labelme.zip # 下载压缩包
!unzip Watermelon87_Semantic_Seg_Labelme.zip >> /dev/null # 解压压缩包
!rm -rf Watermelon87_Semantic_Seg_Labelme.zip # 删除压缩包

--2023-06-09 09:09:27--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/dataset/watermelon/Watermelon87_Semantic_Seg_Labelme.zip
正在连接 172.16.0.13:5848... 已连接。
已发出 Proxy 请求，正在等待回应... 200 OK
长度： 12978831 (12M) [application/zip]
正在保存至: “Watermelon87_Semantic_Seg_Labelme.zip”

Watermelon87_Semant 100%[===================>]  12.38M  26.4MB/s    用时 0.5s    

2023-06-09 09:09:28 (26.4 MB/s) - 已保存 “Watermelon87_Semantic_Seg_Labelme.zip” [12978831/12978831])



## 查看数据集目录结构

In [2]:
import seedir as sd
sd.seedir('Watermelon87_Semantic_Seg_Labelme', style='emoji', depthlimit=1)

📁 Watermelon87_Semantic_Seg_Labelme/
├─📁 images/
└─📁 labelme_jsons/


## 删除系统自动生成的多余文件

### 查看待删除的多余文件

In [3]:
!find . -iname '__MACOSX'

In [4]:
!find . -iname '.DS_Store'

In [5]:
!find . -iname '.ipynb_checkpoints'

./.ipynb_checkpoints


### 删除多余文件

In [6]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

In [7]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

In [8]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

### 验证多余文件已删除

In [9]:
!find . -iname '__MACOSX'

In [10]:
!find . -iname '.DS_Store'

In [11]:
!find . -iname '.ipynb_checkpoints'

## 进入数据集目录

In [12]:
import os
import json
import numpy as np
import cv2
import shutil

from tqdm import tqdm

## 数据集及类别信息

In [13]:
Dataset_Path = 'Watermelon87_Semantic_Seg_Labelme'

## 每个类别的信息及画mask的顺序（按照由大到小，由粗到精的顺序）

In [14]:
# 0-背景，从 1 开始
class_info = [
    {'label':'red', 'type':'polygon', 'color':1},                    # polygon 多段线
    {'label':'green', 'type':'polygon', 'color':2},
    {'label':'white', 'type':'polygon', 'color':3},
    {'label':'seed-black','type':'polygon','color':4},
    {'label':'seed-white','type':'polygon','color':5}
]

## 单张图像labelme转mask函数

In [15]:
def labelme2mask_single_img(img_path, labelme_json_path):
    '''
    输入原始图像路径和labelme标注路径，输出 mask
    '''
    
    img_bgr = cv2.imread(img_path)
    img_mask = np.zeros(img_bgr.shape[:2]) # 创建空白图像 0-背景
    
    with open(labelme_json_path, 'r', encoding='utf-8') as f:
        labelme = json.load(f)
        
    for one_class in class_info: # 按顺序遍历每一个类别
        for each in labelme['shapes']: # 遍历所有标注，找到属于当前类别的标注
            if each['label'] == one_class['label']:
                if one_class['type'] == 'polygon': # polygon 多段线标注

                    # 获取点的坐标
                    points = [np.array(each['points'], dtype=np.int32).reshape((-1, 1, 2))]

                    # 在空白图上画 mask（闭合区域）
                    img_mask = cv2.fillPoly(img_mask, points, color=one_class['color'])

                elif one_class['type'] == 'line' or one_class['type'] == 'linestrip': # line 或者 linestrip 线段标注

                    # 获取点的坐标
                    points = [np.array(each['points'], dtype=np.int32).reshape((-1, 1, 2))]

                    # 在空白图上画 mask（非闭合区域）
                    img_mask = cv2.polylines(img_mask, points, isClosed=False, color=one_class['color'], thickness=one_class['thickness']) 

                elif one_class['type'] == 'circle': # circle 圆形标注

                    points = np.array(each['points'], dtype=np.int32)

                    center_x, center_y = points[0][0], points[0][1] # 圆心点坐标

                    edge_x, edge_y = points[1][0], points[1][1]     # 圆周点坐标

                    radius = np.linalg.norm(np.array([center_x, center_y] - np.array([edge_x, edge_y]))).astype('uint32') # 半径

                    img_mask = cv2.circle(img_mask, (center_x, center_y), radius, one_class['color'], one_class['thickness'])

                else:
                    print('未知标注类型', one_class['type'])
                    
    return img_mask

## labelme转mask-批量

In [16]:
os.chdir(Dataset_Path)
os.mkdir('masks')
os.chdir('images')

In [17]:
for img_path in tqdm(os.listdir()):
    
    try:
    
        labelme_json_path = os.path.join('../', 'labelme_jsons', '.'.join(img_path.split('.')[:-1])+'.json')

        img_mask = labelme2mask_single_img(img_path, labelme_json_path)

        mask_path = img_path.split('.')[0] + '.png'

        cv2.imwrite(os.path.join('../','masks',mask_path), img_mask)
    
    except Exception as E:
        print(img_path, '转换失败', E)
        

100%|██████████| 87/87 [00:01<00:00, 45.33it/s]


## 转换之后的mask保存在`masks`文件夹中

## 重命名和删除文件夹

In [18]:
os.chdir('../')
shutil.move('images', 'img_dir')
shutil.move('masks', 'ann_dir')
!rm -rf labelme_jsons
os.chdir('../')

## 得到最终的语义分割数据集

## 查看数据集目录结构

In [19]:
import seedir as sd
sd.seedir('Watermelon87_Semantic_Seg_Labelme', style='emoji', depthlimit=1)

📁 Watermelon87_Semantic_Seg_Labelme/
├─📁 img_dir/
└─📁 ann_dir/
